# Proyecto de Scrapping de Estadísticas de Videojuegos - Hugo Peralta Muñoz

En este proyecto se desarrollará el *scrapping* de una página web de videojuegos, la cual actúa como base de datos para almacenar cada uno de ellos. La página es la siguiente: <a>https://www.mobygames.com/</a>. Esta es una de las webs más usadas para buscar sobre videojuegos.

El scrapeo se realizará con la biblioteca `BeautifulSoup4` en *Python*. Podrás encontrar información útil de su aplicación y la misma documentación en la página de *Python Packages Index*: <a>https://pypi.org/project/beautifulsoup4/</a>.

## Importación de librerías

In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

## Inicialización de datos 

In [23]:
# Inicializacion de variables
pages = 2

fields = []

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

games_df = pd.DataFrame(columns=fields)

Tendremos almacenadas las URLs que usaremos para acceder a la página y scrapear su contenido.

In [24]:
page_games_url = "https://www.metacritic.com/browse/game/?releaseYearMin=1958&releaseYearMax=2025&page="

## Scrapeo de datos

### Scrapeo de las paginas

Cada página de la base de datos muestra $50$ juegos, scrapearemos primero las entradas de estos videojuegos. 

In [25]:
game_pages = []

for page in range(1, pages + 1):
    game_pages.append(page_games_url + str(page))

game_pages

['https://www.mobygames.com/game/sort:moby_score/page:1/',
 'https://www.mobygames.com/game/sort:moby_score/page:2/']

Una vez tengamos las urls de las páginas, tendremos que sacar las entradas de cada videojuegos.

In [36]:
game_links = []

for page in game_pages:
    response = requests.get(page, headers=header)
    soup = BeautifulSoup(response.content, "html.parser")

    



game_links

None
None


[]

Ahora, nuestra variable `game_links` almacena el enlace a cada juego de forma individual, lo que necesitamos ahora es sacar los datos relevantes que queremos obtener, las cuales serán las variables inicializadas con anterioridad.

In [27]:
rank_counter = 1
for game_url in game_links:

    response = requests.get(game_url)
    soup = BeautifulSoup(response.content, "html.parser")

    

NameError: name 'game_data' is not defined